In [17]:

# Install TA-Lib https://medium.com/pythons-gurus/how-to-properly-install-ta-lib-on-windows-11-for-python-a-step-by-step-guide-13ebb684f4a6

import pandas_ta as pta
import talib as ta
# help(ta.aberration)
help(pta.cci)

Help on function cci in module pandas_ta.momentum.cci:

cci(high, low, close, length=None, c=None, talib=None, offset=None, **kwargs)
    Commodity Channel Index (CCI)

    Commodity Channel Index is a momentum oscillator used to primarily identify
    overbought and oversold levels relative to a mean.

    Sources:
        https://www.tradingview.com/wiki/Commodity_Channel_Index_(CCI)

    Calculation:
        Default Inputs:
            length=14, c=0.015
        SMA = Simple Moving Average
        MAD = Mean Absolute Deviation
        tp = typical_price = hlc3 = (high + low + close) / 3
        mean_tp = SMA(tp, length)
        mad_tp = MAD(tp, length)
        CCI = (tp - mean_tp) / (c * mad_tp)

    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 14
        c (float): Scaling Constant. Default: 0.015
        talib (bool): If TA Lib is installed and t

In [14]:
MyStrategy = pta.Strategy(
    name="xxx",
    ta=[
        {"kind": "ohlc4"},
        {"kind": "sma", "length": 10}
    ]
)

In [18]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Define the ticker symbol and interval
ticker = "AAPL"

data = yf.download(ticker, interval="1d", start="2024-01-01", end="2024-10-01")
# Resample data to create K-line intervals, for example, every 5 days
data.columns = data.columns.droplevel("Ticker")
data.columns
k_interval = '5D'
k_line_data = data.resample(k_interval).agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})
print(k_line_data.head())

[*********************100%***********************]  1 of 1 completed

Price                            Open        High         Low       Close  \
Date                                                                        
2024-01-02 00:00:00+00:00  187.149994  188.440002  180.169998  181.179993   
2024-01-07 00:00:00+00:00  182.089996  187.050003  181.500000  185.589996   
2024-01-12 00:00:00+00:00  186.059998  186.740005  180.929993  183.630005   
2024-01-17 00:00:00+00:00  181.270004  191.949997  180.300003  191.559998   
2024-01-22 00:00:00+00:00  192.300003  196.380005  191.940002  192.419998   

Price                         Volume  
Date                                  
2024-01-02 00:00:00+00:00  275190100  
2024-01-07 00:00:00+00:00  197907600  
2024-01-12 00:00:00+00:00  106047700  
2024-01-17 00:00:00+00:00  194064200  
2024-01-22 00:00:00+00:00  255536900  


In [16]:
data["DragonflyDoji"] = ta.CDLDRAGONFLYDOJI(
    k_line_data['Open'], 
    k_line_data['High'], 
    k_line_data['Low'], 
    k_line_data['Close']
)

In [20]:
# backtest vectorbt
import vectorbt as vbt

hammer = ta.CDLHAMMER(data.Open, data.High, data.Low, data.Close)
hanging_man = ta.CDLHANGINGMAN(data.Open, data.High, data.Low, data.Close)
buys = (hammer ==100)
sells = (hanging_man == -100)
pf = vbt.Portfolio.from_signals(data.Close, buys, sells)
print(pf.stats())



c:\Users\wattx\Documents\Stock\stock_analysis\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\wattx\Documents\Stock\stock_analysis\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\wattx\Documents\Stock\stock_analysis\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\wattx\Documents\Stock\stock_analysis\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2024-01-02 00:00:00+00:00
End                           2024-09-30 00:00:00+00:00
Period                                              188
Start Value                                       100.0
End Value                                     119.79718
Total Return [%]                               19.79718
Benchmark Return [%]                          25.511744
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              13.941478
Max Drawdown Duration                              78.0
Total Trades                                          2
Total Closed Trades                                   2
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                      100.0
Best Trade [%]                                18.823352
Worst Trade [%]                                 